# business Problem

#### Objetivo do Problema
    -- Previsão do primeiro destino que um novo usuario irá escolher.

#### Porque?
**Qual tipo de modelo de negócio do AirBNB?**
    
    -- MarketPlace (Conecta pessoas que oferecem acomodação, com pessoas que estão procurando acomodação).

##### Oferta (Pessoas oferecendo acomodação)

    -- Tamanho do portfolio
    -- Diversidade/densidade de portfolio
    -- Preço medio

##### Demanda (Pessoas procurando acomodação)

    -- Numero de usuarios
    -- LTV (Lifitime Value)
    -- CAC (Client Acquisition Cost)

**Cross Revenue** (FEE * Numero Cliente) - CAC

#### Proposta da Solução

    -- Modelo de predição do primeiro destino de um novo usuario.

##### Tipos de formatos para entrega do projeto.
    -- Predições salva em tabela do banco de dados.
    -- API: Input: Usuario e suas caracteristicas / Output: Usuario e suas caracteristicas com as Predições do destino.

# Ciclo03: Cross Validation

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import preprocessing as pp
from sklearn import metrics as m
from scikitplot import metrics as mt

from keras import models as ml
from keras import layers as l

import warnings
import pickle

warnings.filterwarnings('ignore')

2022-12-26 21:21:16.191319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-26 21:21:16.282785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-26 21:21:16.282798: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-26 21:21:16.302378: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-26 21:21:16.704214: W tensorflow/stream_executor/platform/de

## Paths

In [2]:
path_data = '/home/wellington/Documentos/Projetos/Projetos_ds_pa/airbnb_predict_first_booking/airbnb_predict_first_booking/data/'
path_model = '/home/wellington/Documentos/Projetos/Projetos_ds_pa/airbnb_predict_first_booking/airbnb_predict_first_booking/models/'
path_transformation = '/home/wellington/Documentos/Projetos/Projetos_ds_pa/airbnb_predict_first_booking/airbnb_predict_first_booking/transformation_data/'

# 5.0 Data Preparation 

In [3]:
data_preparation = pd.read_csv(path_data+'ciclo002_Imbalanced_metrics_data_users.csv')

### GetDummies values

In [4]:
#GetDummies values
data_preparation_dummies = pd.get_dummies(data_preparation.drop(['id', 'country_destination'], axis=1))

# Jooin id And country destination
data_preparation_final = pd.concat( [data_preparation[['id', 'country_destination']], data_preparation_dummies], axis=1)

In [5]:
data_preparation_final.head()

,id,country_destination,age,signup_flow,date_account_created_2010-01-02,date_account_created_2010-01-03,date_account_created_2010-01-04,date_account_created_2010-01-07,date_account_created_2010-01-10,date_account_created_2010-01-11,...,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser
0,4ft3gnwmtx,US,56,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,bjjt8pjhuk,other,42,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,87mebub9p4,US,41,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,lsw9q7uk0j,US,46,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0d01nltbrs,US,47,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# 6.0 Feature Selection

In [6]:
X = data_preparation_final.drop(['id', 'country_destination'], axis=1)
y = data_preparation_final['country_destination'].copy()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=32)

In [21]:
ohe = pp.OneHotEncoder()

y_train_nn = ohe.fit_transform( y_train.values.reshape( -1, 1)).toarray()

# 7.0 Machine Learning Model

## Cross-Validation

In [22]:
#Generate K-fold
num_folds = 5

balanced_acc_list = []
kappa_acc_list = []
i = 1 

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=32)

for train_ix, val_ix in kfold.split(x_train, y_train):
    print('Fold Number: {}/{}'.format(i, num_folds))


    #get fold
    x_train_folds = x_train.iloc[train_ix]
    y_train_folds = y_train.iloc[train_ix]

    x_val_fold = x_train.iloc[val_ix]
    y_val_folds = y_train.iloc[val_ix]

    #Target Hot-encoding
    ohe = pp.OneHotEncoder()

    y_train_fold_nn = ohe.fit_transform( y_train_folds.values.reshape( -1, 1)).toarray() #Precisamos transformar nosso y_train neste formato, pois rede neural so aceita dessa forma.

    #Model Definition
    model = ml.Sequential()

    model.add( l.Dense(256, input_dim=x_train.shape[1], activation= 'relu'))
    model.add( l.Dense( 11, activation = 'softmax'))

    #model Compile
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    #Train model
    model.fit( x_train_folds, y_train_fold_nn, epochs=100, batch_size=32, verbose=0)

    #Prediction
    pred_nn = model.predict(x_val_fold)

    #Invert prediction
    yhat_nn = ohe.inverse_transform(pred_nn)

    #Prediction prepare
    y_val_nn = y_val_folds.to_numpy()
    yhat_nn = yhat_nn.reshape(1, -1)[0]

    #Metrics
    
    ##Balanced Accuracy
    balanced_acc_nn = m.balanced_accuracy_score(y_val_nn, yhat_nn)
    balanced_acc_list.append(balanced_acc_nn)

    #Kappa Metric

    kappa_acc_nn = m.cohen_kappa_score(y_val_nn, yhat_nn)
    kappa_acc_list.append(kappa_acc_nn)

    i += 1

Fold Number: 1/5


2022-11-17 20:22:04.025660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 20:22:04.025812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 20:22:04.025860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-17 20:22:04.025900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-17 20:22:04.025939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

341/341 [==============================] - 0s 419us/step
Fold Number: 2/5
341/341 [==============================] - 0s 425us/step
Fold Number: 3/5
341/341 [==============================] - 0s 409us/step
Fold Number: 4/5
341/341 [==============================] - 0s 435us/step
Fold Number: 5/5
341/341 [==============================] - 0s 411us/step


In [26]:
print('Avg Balanced Accuracy: {} +/- {}'.format(np.round( np.mean(balanced_acc_list), 2), np.round(np.std(balanced_acc_list),4)))

print('Avg Kappa Accuracy: {} +/- {}'.format(np.round( np.mean(kappa_acc_list),4), np.round( np.std(kappa_acc_list),4)))

Avg Balanced Accuracy: 0.09 +/- 0.0004
Avg Kappa Accuracy: 0.0023 +/- 0.0012
